In [1]:
################################################################
# For documentation on the Zip_time_series.csv file, refer to  #
# https://www.kaggle.com/zillow/zecon#Zip_time_series.csv      #
# The file may also be downloaded from here                    #
################################################################

# Dependencies
import pandas as pd
import numpy as np
from usefuls import atx_zip_codes, census_key, zipcode_tabulation_area
import requests
from census import Census

In [2]:
# Import file as dataframe and preview
# .csv file needs to be in same directory as this file.
df = pd.read_csv('Zip_time_series.csv')

df.head()

,Date,RegionName,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,MedianListingPricePerSqft_AllHomes,...,ZHVI_BottomTier,ZHVI_CondoCoop,ZHVI_MiddleTier,ZHVI_SingleFamilyResidence,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental
0,1996-04-30,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,68700.0,67000.0,101900.0,107000.0,124800.0,NaN,NaN,NaN,NaN,NaN
1,1996-04-30,1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,97000.0,81300.0,135200.0,145800.0,213600.0,NaN,NaN,NaN,NaN,NaN
2,1996-04-30,1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85400.0,NaN,101200.0,101200.0,125700.0,NaN,NaN,NaN,NaN,NaN
3,1996-04-30,1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,97900.0,87600.0,124000.0,125000.0,162100.0,NaN,NaN,NaN,NaN,NaN
4,1996-04-30,1008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,81100.0,NaN,109300.0,109300.0,148100.0,NaN,NaN,NaN,NaN,NaN


In [3]:
# Rename RegionName to zipcode because that's what it is
df = df.rename(columns={'RegionName': 'zip_code'})

In [4]:
# Filter out all non-Austin area zipcodes
df = df[[(x in atx_zip_codes) for x in df['zip_code']]]

In [5]:
# Add year column to dataframe
df['year'] = df['Date'].apply(lambda s : int(s[:4]))

In [6]:
# Filter out all years before 2009
df = df[df['year'] >= 2009]

In [7]:
# Now, take out everything except year, zipcode, and Zillow Home Value Index for all homes 
mask = ['year', 'zip_code', 'ZHVI_AllHomes']
df = df[mask]

In [8]:
df = df.dropna()

In [9]:
# Group data by year, zipcode, find the mean ZHVI per year per zipcode
df = round(df.groupby(['year', 'zip_code']).ZHVI_AllHomes.mean(),2).to_frame()

In [10]:
# Preview
df.head()

ZHVI_AllHomes
year zip_code               
2009 78610         180533.33
     78613         190950.00
     78617         116075.00
     78641         153816.67
     78664         138175.00

In [11]:
# Save to csv
#df.to_csv('atx_mean_zhvi_2009-2017.csv')

In [12]:
crime_df = pd.read_csv('Crime_Reports (2).csv', dtype={'APD District': str}, low_memory=False)

In [13]:
len(crime_df)

2112338

In [14]:
crime_df.columns

Index(['Incident Number', 'Highest Offense Description',
       'Highest Offense Code', 'Family Violence', 'Occurred Date Time',
       'Occurred Date', 'Occurred Time', 'Report Date Time', 'Report Date',
       'Report Time', 'Location Type', 'Address', 'Zip Code',
       'Council District', 'APD Sector', 'APD District', 'PRA', 'Census Tract',
       'Clearance Status', 'Clearance Date', 'UCR Category',
       'Category Description', 'X-coordinate', 'Y-coordinate', 'Latitude',
       'Longitude', 'Location'],
      dtype='object')

In [15]:
crime_df = crime_df.drop(['Incident Number', 'Occurred Date Time', 'Report Date Time',
                          'Location Type', 'Council District','Council District', 'APD Sector',
                          'APD District', 'PRA', 'Census Tract', 'X-coordinate', 'Y-coordinate', 'Location',
                          'Report Date', 'Occurred Time', 'Report Time', 'UCR Category',
                          'Category Description', 'Clearance Date', 'Highest Offense Code',
                          'Address', 'Latitude', 'Longitude', 'Family Violence'], axis=1)

In [16]:
crime_df.head()

,Highest Offense Description,Occurred Date,Zip Code,Clearance Status
0,AGG ROBBERY/DEADLY WEAPON,09/14/2017,78752.0,O
1,BURGLARY OF VEHICLE,10/11/2014,78701.0,N
2,FORGERY AND PASSING,02/11/2015,78702.0,N
3,POSS OF DRUG PARAPHERNALIA,10/01/2018,78759.0,N
4,POSS OF DRUG PARAPHERNALIA,11/23/2003,78701.0,C


In [17]:
crime_df = crime_df.rename(index=str, columns={"Highest Offense Description": "highest_offense_description",
                                               "Occurred Date": "occurred_date",
                                               "Address": "address",
                                               "Zip Code": "zip_code",
                                               "Clearance Status": "clearance_status"})
crime_df.head()

,highest_offense_description,occurred_date,zip_code,clearance_status
0,AGG ROBBERY/DEADLY WEAPON,09/14/2017,78752.0,O
1,BURGLARY OF VEHICLE,10/11/2014,78701.0,N
2,FORGERY AND PASSING,02/11/2015,78702.0,N
3,POSS OF DRUG PARAPHERNALIA,10/01/2018,78759.0,N
4,POSS OF DRUG PARAPHERNALIA,11/23/2003,78701.0,C


In [18]:
crime_df['year'] = crime_df['occurred_date'].apply(lambda d : int(d[-4:]))
crime_df.head()

,highest_offense_description,occurred_date,zip_code,clearance_status,year
0,AGG ROBBERY/DEADLY WEAPON,09/14/2017,78752.0,O,2017
1,BURGLARY OF VEHICLE,10/11/2014,78701.0,N,2014
2,FORGERY AND PASSING,02/11/2015,78702.0,N,2015
3,POSS OF DRUG PARAPHERNALIA,10/01/2018,78759.0,N,2018
4,POSS OF DRUG PARAPHERNALIA,11/23/2003,78701.0,C,2003


In [19]:
crime_df = crime_df[crime_df.year >= 2009]
len(crime_df)

1275108

In [20]:
crime_df = crime_df[crime_df.year <= 2017]
len(crime_df)

1157288

In [21]:
crime_df.head()

,highest_offense_description,occurred_date,zip_code,clearance_status,year
0,AGG ROBBERY/DEADLY WEAPON,09/14/2017,78752.0,O,2017
1,BURGLARY OF VEHICLE,10/11/2014,78701.0,N,2014
2,FORGERY AND PASSING,02/11/2015,78702.0,N,2015
5,AUTO THEFT,07/07/2015,78759.0,N,2015
6,BURGLARY OF RESIDENCE,05/11/2013,78741.0,N,2013


In [22]:
crime_df.count()

highest_offense_description    1157288
occurred_date                  1157288
zip_code                       1151277
clearance_status               1017109
year                           1157288
dtype: int64

In [23]:
crime_df = crime_df.dropna()

In [24]:
len(crime_df)

1012269

In [25]:
crime_df.count()

highest_offense_description    1012269
occurred_date                  1012269
zip_code                       1012269
clearance_status               1012269
year                           1012269
dtype: int64

In [26]:
#crime_df['zip_code'] = crime_df.zip_code.astype(int).astype(str)

In [27]:
crime_df.dtypes

highest_offense_description     object
occurred_date                   object
zip_code                       float64
clearance_status                object
year                             int64
dtype: object

In [28]:
crime_df.head()

,highest_offense_description,occurred_date,zip_code,clearance_status,year
0,AGG ROBBERY/DEADLY WEAPON,09/14/2017,78752.0,O,2017
1,BURGLARY OF VEHICLE,10/11/2014,78701.0,N,2014
2,FORGERY AND PASSING,02/11/2015,78702.0,N,2015
5,AUTO THEFT,07/07/2015,78759.0,N,2015
6,BURGLARY OF RESIDENCE,05/11/2013,78741.0,N,2013


In [29]:
#atx_zip_codes2 = ['78610', '78613', '78617', '78641', '78664', '78681', '78701', '78702', '78703', '78704', '78717',
                 #'78721', '78722', '78723', '78725', '78726', '78727', '78728', '78729', '78731', '78733', '78735',
                 #'78736', '78738', '78739', '78741', '78744', '78745', '78746', '78748', '78749', '78751', '78752',
                 #'78753', '78756', '78757', '78758', '78759']
#crime_df = crime_df[crime_df['zip_code'].isin(atx_zip_codes2)]
#crime_df.head()

In [30]:
crime_df = crime_df[[(x in atx_zip_codes) for x in crime_df['zip_code']]]

In [31]:
crime_df.head()

,highest_offense_description,occurred_date,zip_code,clearance_status,year
0,AGG ROBBERY/DEADLY WEAPON,09/14/2017,78752.0,O,2017
1,BURGLARY OF VEHICLE,10/11/2014,78701.0,N,2014
2,FORGERY AND PASSING,02/11/2015,78702.0,N,2015
5,AUTO THEFT,07/07/2015,78759.0,N,2015
6,BURGLARY OF RESIDENCE,05/11/2013,78741.0,N,2013
